In [3]:
%load_ext autoreload
%autoreload 2

import os
import time
import numpy as np
import torch
from tqdm import trange

from game.flappy_bird import FlappyBirdEnv
from agent.dqn_agent import DQNAgent, DQNConfig
CKPT_PATH = "agent/flappy_dqn_live.pth"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

env = FlappyBirdEnv(pipe_gap=160, pipe_speed=2.0, seed=42)
state_dim = 5
action_dim = 2

cfg = DQNConfig(
    start_learning_after=1000,  # earlier learning
    eps_decay_steps=20000,    
	state_dim=state_dim,
	action_dim=action_dim,
	gamma=0.99,
	lr=1e-3,
	batch_size=64,
	replay_size=50_000,
	
	target_update_freq=1_000,
	eps_start=1.0,
	eps_end=0.05,
	
	gradient_clip_norm=5.0,
)

agent = DQNAgent(cfg)
print("Using device:", agent.cfg.device)

Using device: cpu


In [8]:
import numpy as np

def simple_heuristic_action(state):
    bird_y, bird_vel, next_x, top, bottom = state
    target_y = (top + bottom) / 2.0
    return 1 if bird_y > target_y + 0.03 else 0

warm_steps = 5000
s = env.reset()
for _ in range(warm_steps):
    if np.random.rand() < 0.2:
        a = np.random.randint(2)
    else:
        a = simple_heuristic_action(s)
    ns, r, d, info = env.step(a)
    agent.store(s, a, r, ns, d)
    s = ns if not d else env.reset()
print("Replay warm-started with heuristic + random steps.")

Replay warm-started with heuristic + random steps.


In [9]:
num_episodes = 500
max_steps_per_ep = 10_000

moving_avg_window = 50
ep_returns = []
losses = []

for ep in trange(num_episodes, desc="Training"):
	state = env.reset()
	ep_return = 0.0

	for t in range(max_steps_per_ep):
		action = agent.select_action(state)
		next_state, reward, done, info = env.step(action)

		agent.store(state, action, reward, next_state, done)
		loss = agent.train_step()
		if loss is not None:
			losses.append(loss)

		# autosave every 1k steps so a live viewer can reload
		if agent.total_steps % 1000 == 0:
			agent.save(CKPT_PATH)

		ep_return += reward
		state = next_state
		if done:
			break

	ep_returns.append(ep_return)
	if (ep + 1) % 10 == 0:
		recent = ep_returns[-moving_avg_window:]
		mavg = np.mean(recent) if recent else 0.0
		print(f"Ep {ep+1}/{num_episodes} | Return: {ep_return:.1f} | 50-ep avg: {mavg:.1f} | Eps: {agent.epsilon:.3f}")

# final save
agent.save(CKPT_PATH)
print("Training finished. Saved:", CKPT_PATH)

Training:   2%|▏         | 11/500 [00:01<01:10,  6.98it/s]

Ep 10/500 | Return: -62.0 | 50-ep avg: -63.7 | Eps: 0.982


Training:   4%|▍         | 21/500 [00:03<01:06,  7.25it/s]

Ep 20/500 | Return: -65.0 | 50-ep avg: -63.1 | Eps: 0.964


Training:   6%|▌         | 31/500 [00:04<01:01,  7.66it/s]

...

Ep 480/500 | Return: 43.0 | 50-ep avg: 24.5 | Eps: 0.050


Training:  98%|█████████▊| 490/500 [01:47<00:06,  1.49it/s]

Ep 490/500 | Return: 43.0 | 50-ep avg: 35.6 | Eps: 0.050


Training: 100%|██████████| 500/500 [01:52<00:00,  4.43it/s]

Ep 500/500 | Return: -20.0 | 50-ep avg: 34.2 | Eps: 0.050
Training finished. Saved: agent/flappy_dqn_live.pth


In [10]:
save_path = "agent/flappy_dqn.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
agent.save(save_path)
print("Saved:", save_path)

Saved: agent/flappy_dqn.pth


In [11]:

env.close()
state = env.reset()

total_reward = 0.0
for _ in range(5000):
	# Greedy action for evaluation (no epsilon)
	with torch.no_grad():
		state_t = torch.from_numpy(state).float().unsqueeze(0).to(agent.cfg.device)
		q_vals = agent.q_net(state_t)
		action = int(torch.argmax(q_vals, dim=1).item())

	next_state, reward, done, info = env.step(action)
	env.render(fps=60)
	total_reward += reward
	state = next_state
	if done:
		break

print("Eval episode reward:", total_reward)
env.close()

Eval episode reward: 74.0
